# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.3.3


In [2]:
!pip freeze | grep tensorflow

tensorflow==2.19.0
tensorflow-datasets==4.9.7
tensorflow-metadata==1.16.1


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2025-05-22 15:28:12.176446: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-22 15:28:12.344879: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-22 15:28:12.512964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747920492.632875    2681 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747920492.669251    2681 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747920492.930880    2681 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/NLP/layers_embedding.png" alt="Joint learning of embedding and RNN" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/NLP/word2vec_representation.png" alt="Training of RNN and embedding independently" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/NLP/word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [4]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [15]:
wv['germany']

array([-0.01259061,  0.10897601, -0.03541998,  0.14515454, -0.03285385,
       -0.21428728,  0.07928195,  0.32540518, -0.12989151, -0.13521509,
        0.05397818, -0.1917811 ,  0.05130879,  0.11838851,  0.00855492,
       -0.06918262,  0.14344034, -0.07701989, -0.04957432, -0.23032683,
        0.08620376,  0.02644371,  0.15865871, -0.12482186, -0.01105135,
        0.00553806, -0.15416706, -0.04973375, -0.06478898, -0.0117502 ,
        0.09898517,  0.00270209,  0.08067389, -0.11524189, -0.14111122,
        0.08576984,  0.08506197, -0.08308662, -0.1514504 , -0.17136   ,
       -0.03700036, -0.12652302, -0.12203512,  0.02334025,  0.07869741,
        0.027787  , -0.09930664, -0.07569452,  0.10504513,  0.14543156,
       -0.01989167, -0.12412392, -0.18037644, -0.02635814, -0.07090568,
        0.03672115,  0.12987715, -0.02429155, -0.0545173 ,  0.01738367,
        0.02086783, -0.0743294 ,  0.04546538,  0.07240435, -0.1031701 ,
        0.13981907, -0.04025728,  0.02626171, -0.21264847,  0.11

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [16]:
print(wv['germany'].shape)


(100,)


🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [20]:
# Check most similar words to 'germany'
print(wv.most_similar('germany'))


[('guinea', 0.9817646145820618), ('2001', 0.9802541136741638), ('gamera', 0.9777314066886902), ('cold', 0.9777153134346008), ('leonard', 0.9771721959114075), ('1980', 0.9770714044570923), ('ii', 0.9769440293312073), ('absolute', 0.9765051007270813), ('17', 0.976080596446991), ('naval', 0.9756405353546143)]


In [21]:
print(wv.most_similar('india'))

[('culture', 0.9872554540634155), ('naval', 0.9858789443969727), ('tied', 0.9855062365531921), ('walls', 0.984674870967865), ('species', 0.9846397042274475), ('happiness', 0.9841641187667847), ('ship', 0.9839056730270386), ('chamber', 0.9838953614234924), ('packed', 0.9833608865737915), ('matthau', 0.9824745059013367)]


📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [22]:
print(wv.similar_by_vector('india'))

[('culture', 0.9872554540634155), ('naval', 0.9858789443969727), ('tied', 0.9855062365531921), ('walls', 0.984674870967865), ('species', 0.9846397042274475), ('happiness', 0.9841641187667847), ('ship', 0.9839056730270386), ('chamber', 0.9838953614234924), ('packed', 0.9833608865737915), ('matthau', 0.9824745059013367)]


# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [29]:
wv['germany'] - wv['india']


array([-1.8227987e-02,  1.0581054e-02,  4.5964494e-02,  3.1593062e-02,
        1.7846934e-03, -6.4170435e-02,  8.7895423e-02,  6.1400950e-02,
       -3.3302255e-02, -2.7431145e-02,  3.0003792e-02, -1.6557023e-02,
        4.4200271e-03, -6.5019950e-03,  7.4013234e-03,  3.2767683e-02,
        1.9627988e-02,  2.1143124e-02, -2.5897263e-02, -3.7347078e-02,
       -1.8285006e-02,  4.4556700e-02, -1.3010412e-02,  1.2605637e-02,
       -1.1572538e-02,  1.4520551e-02,  2.3665577e-03, -2.8671417e-02,
        2.5493942e-02, -1.6224977e-02, -1.3280414e-02, -1.1190812e-02,
        6.8947375e-03,  4.4505127e-02, -1.2437791e-02,  3.2861862e-02,
        4.7460794e-03, -1.4708325e-02, -9.7848028e-03, -1.1555448e-02,
        2.8247558e-02,  1.1059046e-02,  4.3849796e-03, -4.4164144e-02,
        7.2652102e-04,  1.2318591e-02,  4.7510117e-03,  4.5095384e-04,
       -1.8128276e-02,  4.8203021e-03, -3.4632180e-02, -2.7462155e-02,
       -1.3853893e-02, -2.1739751e-03, -2.8535649e-03, -2.9220328e-02,
      

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [31]:
res = wv['good']-wv['bad']+wv['stupid']
print(res)

[-0.07319799 -0.08192265  0.25728452 -0.00549683  0.084134   -1.055527
 -0.38375267  0.96810323 -0.82646006 -0.980011   -0.06371433 -0.64923036
  0.45455718  0.287198    0.02552099 -0.37216562  0.06024289 -0.20910937
 -0.22099082 -0.94026154  0.41947013  0.15370622  0.6776504  -0.15451182
 -0.0754095  -0.24708644 -0.4656643   0.4080537  -0.36283067 -0.46379378
  0.11842424  0.2063068   0.50628424 -0.31047693 -0.4069112   0.26440632
  0.46300662 -0.3017544  -0.33998182  0.05125105 -0.07403755 -0.4883195
 -0.40367386  0.16172382  0.5824053  -0.3014261   0.3531086  -0.4297876
 -0.28623593  0.42790037 -0.04284374 -0.16454421 -0.73710984  0.42300236
  0.02700239  0.42088002  0.18963303  0.05300869 -0.29741216  0.06858104
  0.16341689  0.16211003 -0.02859178 -0.08299357 -0.1910814   0.4705549
  0.08129907 -0.10774717  0.32296517  0.324759   -0.5260055  -0.02630785
  0.35092822  0.43844658  0.27033928 -0.0097027  -0.01966489 -0.21588317
 -0.02148321  0.14976458  0.39929244 -0.24190737 -0.5750

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [33]:
print(wv.similar_by_vector(res))

[('nice', 0.7872844338417053), ('good', 0.7430687546730042), ('decent', 0.7390629649162292), ('great', 0.733694314956665), ('such', 0.7264269590377808), ('hailed', 0.7223927974700928), ('successful', 0.7219948768615723), ('written', 0.7202735543251038), ('potential', 0.7142477035522461), ('delivered', 0.7101901769638062)]


Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [ ]:
# YOUR CODE HERE

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [34]:
from gensim.models import Word2Vec

# Train a new model with vector_size = 200
word2vec_2 = Word2Vec(sentences=X_train, vector_size=200)

# Get the word vectors
wv2 = word2vec_2.wv


In [36]:
# Choose a word that exists in your training data
print(wv2['germany'].shape)
print(wv2['india'].shape)

(200,)
(200,)


❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [37]:
# Size of learned vocabulary
vocab_size = len(word2vec.wv.key_to_index)
print("Size of learned vocabulary:", vocab_size)


Size of learned vocabulary: 8006


In [38]:
# Flatten all words from all sentences
all_words = [word for sentence in X_train for word in sentence]

# Get the set of unique words
unique_words = set(all_words)

print("Number of unique words in X_train:", len(unique_words))


Number of unique words in X_train: 30419


There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [39]:
# You already have this (default min_count=5)
word2vec = Word2Vec(sentences=X_train, vector_size=100)  # word2vec_1
word2vec_2 = Word2Vec(sentences=X_train, vector_size=200)  # Different vector size

# New model with higher min_count (e.g., 10)
word2vec_3 = Word2Vec(sentences=X_train, vector_size=100, min_count=10)

# New model with lower min_count (e.g., 2)
word2vec_4 = Word2Vec(sentences=X_train, vector_size=100, min_count=2)


In [40]:
vocab_1 = len(word2vec.wv.key_to_index)
vocab_2 = len(word2vec_2.wv.key_to_index)
vocab_3 = len(word2vec_3.wv.key_to_index)
vocab_4 = len(word2vec_4.wv.key_to_index)

print("Vocabulary size for word2vec (default min_count=5):", vocab_1)
print("Vocabulary size for word2vec_2 (default min_count=5, vector_size=200):", vocab_2)
print("Vocabulary size for word2vec_3 (min_count=10):", vocab_3)
print("Vocabulary size for word2vec_4 (min_count=2):", vocab_4)


Vocabulary size for word2vec (default min_count=5): 8006
Vocabulary size for word2vec_2 (default min_count=5, vector_size=200): 8006
Vocabulary size for word2vec_3 (min_count=10): 4503
Vocabulary size for word2vec_4 (min_count=2): 16729


Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [41]:
from gensim.models import Word2Vec

word2vec_5 = Word2Vec(sentences=X_train, vector_size=100, window=2)


In [42]:
print("Vocabulary size for word2vec_5 (window=2):", len(word2vec_5.wv.key_to_index))


Vocabulary size for word2vec_5 (window=2): 8006


The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/NLP/word2vec_representation.png" alt="Word2Vec and RNN" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [53]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    pass  # YOUR CODE HERE

### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

AssertionError: 

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [54]:
import numpy as np

def embedding(word2vec, sentences):
    wv = word2vec.wv
    embedded_sentences = []

    for sentence in sentences:
        # Only keep words that are in the vocabulary
        vectors = [wv[word] for word in sentence if word in wv]
        
        # Convert to a matrix (num_words x vector_size)
        if vectors:
            sentence_matrix = np.vstack(vectors)
        else:
            # If none of the words are in vocab, return a zero row
            sentence_matrix = np.zeros((1, wv.vector_size))
        
        embedded_sentences.append(sentence_matrix)

    return embedded_sentences
X_train_embedded = embedding(word2vec, X_train)
X_test_embedded = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [60]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad the sequences
X_train_pad= pad_sequences(X_train_embedded, dtype='int32', padding='post')
X_test_pad = pad_sequences(X_train_embedded, dtype='int32', padding='post')

# Check the shape after padding
print(f"Padded training shape: {X_train_pad.shape}")
print(f"Padded test shape: {X_test_pad.shape}")


Padded training shape: (2500, 1082, 100)
Padded test shape: (2500, 1082, 100)


In [61]:
### YOUR CODE HERE

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)



🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
